In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from io import StringIO

/Users/rohitramaswamy/Desktop/programming-stuff/sports-data-scraping/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}


In [4]:

year = 2006
df = pd.DataFrame() 
url = 'https://www.teamrankings.com/nba/player-stat/fouls-technical?rate=season-totals&season_id=204'
try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Check if the request was successful

    # Parse the HTML content
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all tables in the page
    tables = soup.find_all('table')

    # Convert the first table to a pandas DataFrame
    if tables:
        print(len(tables))
        df = pd.DataFrame(pd.read_html(StringIO(str(tables[0])))[0])[["Player"]].head(25)
 


        df[f'{year}'] = 0
        # df = df[['Player', 'Goals']]
        
        print(df)
        # if 'Player' in df.columns and 'Runs' in df.columns:
        #     df = df[['Player', 'Wkts']].head(15)
        #     # df.rename(columns={'Wkts': f'{year}'}, inplace=True)
        #     # current_df = pd.merge(current_df, df, on='Player', how='outer')

        # else:
        #     print(f"'Player' or 'Runs' column not found for the year {year}")
    else:
        print("No tables found on the page.")
except requests.HTTPError as e:
    print(f"HTTP error occurred: {e.response.status_code} - {e.response.reason}")
except Exception as e:
    print(f"An error occurred: {str(e)}")



1
               Player  2006
0     Rasheed Wallace     0
1    Richard Hamilton     0
2     Stephen Jackson     0
3         Kobe Bryant     0
4         Baron Davis     0
5         Mikki Moore     0
6   Amar'e Stoudemire     0
7       Kevin Garnett     0
8      Gerald Wallace     0
9     Carmelo Anthony     0
10      Allen Iverson     0
11      Dirk Nowitzki     0
12    Jermaine O'Neal     0
13         Lamar Odom     0
14        Ricky Davis     0
15    Antonio McDyess     0
16          Raja Bell     0
17         Tim Duncan     0
18         Eddy Curry     0
19      Desmond Mason     0
20     Tyson Chandler     0
21       Kirk Hinrich     0
22   Chauncey Billups     0
23     Jamaal Tinsley     0
24      Zach Randolph     0


In [5]:

# Initialize current_df correctly
current_df = pd.DataFrame()
# Function to remove text within parentheses
end_year = 2025
current_year = year + 1

for i in range(end_year - current_year):

    url = f'https://www.teamrankings.com/nba/player-stat/fouls-technical?rate=season-totals&season_id={204 + (current_year - 2007)}'
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Check if the request was successful

        # Parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find all tables in the page
        tables = soup.find_all('table')

        # Convert the first table to a pandas DataFrame
        if tables:
                   

            current_df = pd.DataFrame(pd.read_html(StringIO(str(tables[0])))[0])[["Player", "Value"]].head(25)
            current_df.rename(columns={'Value': f'{current_year}'}, inplace=True)

           
        else:
            print("No tables found on the page.")
    except requests.HTTPError as e:
        print(f"HTTP error occurred: {e.response.status_code} - {e.response.reason}")
    except Exception as e:
        print(f"An error occurred: {str(e)}")
    
    df = pd.merge(df, current_df, on='Player', how='outer')
    df = df.fillna(0)
    


     # Add the goals
    df[f'{current_year}'] = df[f'{current_year - 1}'] + df[f'{current_year}'] 
    

    current_year = current_year + 1

   
    




print(df)




KeyboardInterrupt: 

In [46]:
def strip_non_letters(name):
    # Create an empty list to store only the letters
    filtered_name = [char for char in name if char.isalpha()]
    # Join the list into a string
    stripped_name = ''.join(filtered_name)


    return stripped_name

def get_image(name):
    name_list = list(name.split(" "))
    name_list = [strip_non_letters(name) for name in name_list]
    last_name = ""
    if(len(name_list[1]) < 5):
        last_name = name_list[1]
    else:
        last_name = name_list[1][:5]
    
    # Using indexing sequence
    first_name = name_list[0][:2]
    format = (last_name + first_name + "01").lower()
    return(f"https://www.basketball-reference.com/req/202106291/images/headshots/{format}.jpg")


df["Image"] = df["Player"].apply(get_image)

excel_file = "tech_fouls.xlsx" 
 
# Write DataFrame to Excel 
df.to_excel(excel_file, index=False) 

In [10]:
import requests
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects, RequestException
import time

url = "https://www.teamrankings.com/nba/player-stat/fouls-technical?rate=season-totals&season_id=204"

def fetch_data(url, retries=3, delay=5):
    for i in range(retries):
        try:
            response = requests.get(url)
            response.raise_for_status()
            return response.text
        except ConnectionError:
            print(f"Attempt {i+1} of {retries}: Failed to connect to the server. Retrying in {delay} seconds...")
            time.sleep(delay)
        except Timeout:
            print(f"Attempt {i+1} of {retries}: The request timed out. Retrying in {delay} seconds...")
            time.sleep(delay)
        except TooManyRedirects:
            print("Too many redirects. The URL might be incorrect.")
            break
        except RequestException as e:
            print(f"An error occurred: {e}")
            break
    return None

data = fetch_data(url)
if data:
    # Process the data here
    print("Data fetched successfully!")
else:
    print("Failed to fetch data after multiple attempts.")


Attempt 1 of 3: Failed to connect to the server. Retrying in 5 seconds...
Attempt 2 of 3: Failed to connect to the server. Retrying in 5 seconds...
Attempt 3 of 3: Failed to connect to the server. Retrying in 5 seconds...
Failed to fetch data after multiple attempts.
